# Ejercicio QKD

Mark Albrand - 21004

## Simulación del protocolo BB84

In [139]:
import random
import pandas as pd

In [140]:
# Simbolizamos bases: 'R' (Recta ↕), 'D' (Diagonal ↗)
BASES = ['R', 'D']
BITS = [0, 1]

In [141]:
def choose_bit():
    return random.choice(BITS)

In [142]:
def choose_base():
    return random.choice(BASES)

In [143]:
def encode_photon(bit, base):
    # Simula el fotón como una combinación del bit y la base
    return (bit, base)

In [144]:
def measure_photon(photon, measurement_base):
    original_bit, original_base = photon
    if original_base == measurement_base:
        return original_bit  # base correcta, se mide el bit exacto
    else:
        return random.choice(BITS)  # base incorrecta, se mide aleatoriamente

In [145]:
def simulate_bb84(n=15, eve_present=True, eve_probability=0.3):
    results = []

    for i in range(n):
        alice_bit = choose_bit()
        alice_base = choose_base()
        photon = encode_photon(alice_bit, alice_base)

        # Eve intercepta aleatoriamente
        if eve_present and random.random() < eve_probability:
            eve_base = choose_base()
            eve_bit = measure_photon(photon, eve_base)
            photon = encode_photon(eve_bit, eve_base)  # Eve reenvía un nuevo fotón

        bob_base = choose_base()
        bob_bit = measure_photon(photon, bob_base)
        bases_match = alice_base == bob_base
        use_bit = 'sí' if bases_match else 'no'

        results.append({
            'N°': i+1,
            'Bit de Alice': alice_bit,
            'Base de Alice': '↕' if alice_base == 'R' else '↗',
            'Fotón enviado (codificado)': alice_bit,
            'Base de Bob': '↕' if bob_base == 'R' else '↗',
            'Bit generado': bob_bit,
            '¿Bases coinciden?': 'sí' if bases_match else 'no',
            '¿Usar bit?': use_bit
        })

    return pd.DataFrame(results)

In [146]:
# Ejecutar simulación
df = simulate_bb84(n=15, eve_present=True, eve_probability=0.3)

df

,N°,Bit de Alice,Base de Alice,Fotón enviado (codificado),Base de Bob,Bit generado,¿Bases coinciden?,¿Usar bit?
0,1,0,↕,0,↗,1,no,no
1,2,0,↗,0,↗,0,sí,sí
2,3,0,↕,0,↕,1,sí,sí
3,4,0,↗,0,↕,1,no,no
4,5,0,↗,0,↕,0,no,no
5,6,0,↗,0,↗,0,sí,sí
6,7,1,↕,1,↗,1,no,no
7,8,1,↕,1,↗,0,no,no
8,9,0,↗,0,↗,0,sí,sí
9,10,1,↕,1,↕,1,sí,sí


In [150]:
# Análisis
usable_bits = df[df['¿Usar bit?'] == 'sí']
num_usable = len(usable_bits)
total = len(df)
percentage = (num_usable / total) * 100

print(f"¿Cuántos bits finales obtuvieron de la clave?: {num_usable}")
print(f"¿Qué porcentaje representa respecto al total?: {percentage:.2f}%")

¿Cuántos bits finales obtuvieron de la clave?: 9
¿Qué porcentaje representa respecto al total?: 60.00%


### ¿Qué pasaría si Eve interceptara los fotones y usara bases incorrectas?
Si Eve intercepta y mide los fotones con la base incorrecta, puede alterar el estado del fotón al reenviarlo a Bob. Esto introduce errores en los bits que Bob mide, incluso cuando su base coincide con la de Alice. Como resultado, la clave generada tendrá errores.

### ¿Cómo se puede detectar su presencia?
Después de establecer la clave, Alice y Bob pueden comparar una muestra aleatoria de los bits generados (por un canal público).
Si detectan una tasa de error superior al esperado, concluyen que hubo una interferencia (presencia de un espía).

Ejemplo de prueba: Comparar 25% de la clave públicamente. Si hay muchos errores → ¡Alguien (Eve) está espiando!

### ¿Qué ventajas y desventajas tiene este protocolo frente a otros cifrados tradicionales?

✅ Ventajas:

- Seguridad garantizada por la física cuántica, no por problemas matemáticos difíciles.
- Detección de espionaje: cualquier intento de interceptar la clave altera los resultados y puede ser detectado.
- Es invulnerable a ataques de computadoras cuánticas si se usa correctamente.

⚠️ Desventajas:

- Requiere tecnología cuántica costosa y delicada (emisores y detectores de fotones).
- Tiene limitaciones prácticas como la distancia máxima y la tasa de transmisión.
- No reemplaza por completo el cifrado clásico, se utiliza principalmente para distribuir claves seguras, no para cifrar datos directamente.